# Chapter 02-01: Sequential API

## 학습 목표
- Sequential 모델로 레이어를 쌓는 법을 이해한다
- model.summary()로 파라미터 수를 해석한다
- MNIST 손글씨 분류 전체 파이프라인을 구현한다

## 목차
1. Sequential 모델 생성
2. model.summary() 해석
3. MNIST 분류 실습
4. 결과 시각화

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print("TensorFlow 버전:", tf.__version__)

## 수학적 기초

완전 연결 레이어의 연산:
$$\mathbf{y} = f(\mathbf{W}\mathbf{x} + \mathbf{b})$$

- $\mathbf{W}$: 가중치 행렬, shape $(n_{out}, n_{in})$
- $\mathbf{b}$: 편향 벡터, shape $(n_{out},)$
- $f$: 활성화 함수

**파라미터 수** (Dense 레이어):
$$\text{params} = n_{in} \times n_{out} + n_{out}$$

예시: Dense(784 → 128) = $784 \times 128 + 128 = 100,480$

## 1. Sequential 모델 생성

Sequential API는 레이어를 **순서대로 쌓을 때** 사용한다.

In [ ]:
# ── 방법 1: 생성자에 리스트로 레이어를 전달 ──────────────────────────────
model_v1 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(784,)),  # 입력층 + 첫 번째 은닉층
    tf.keras.layers.Dense(32, activation='relu'),                       # 두 번째 은닉층
    tf.keras.layers.Dense(10, activation='softmax')                     # 출력층 (10개 클래스)
])
print("방법 1 — 생성자 리스트로 생성:")
print("레이어 수:", len(model_v1.layers))

# ── 방법 2: 빈 모델을 만든 뒤 .add()로 레이어를 하나씩 추가 ──────────────
model_v2 = tf.keras.Sequential()                                        # 빈 Sequential 모델 생성
model_v2.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(784,)))  # 첫 번째 레이어 추가
model_v2.add(tf.keras.layers.Dense(32, activation='relu'))              # 두 번째 레이어 추가
model_v2.add(tf.keras.layers.Dense(10, activation='softmax'))           # 출력 레이어 추가
print("\n방법 2 — .add()로 순차 추가:")
print("레이어 수:", len(model_v2.layers))

## 2. model.summary() 해석

In [ ]:
# summary 출력을 위한 예시 모델 생성
summary_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,),
                          name='hidden_1'),   # 파라미터: 784*128 + 128 = 100,480
    tf.keras.layers.Dense(64, activation='relu',
                          name='hidden_2'),   # 파라미터: 128*64 + 64  = 8,256
    tf.keras.layers.Dense(10, activation='softmax',
                          name='output')      # 파라미터: 64*10 + 10   = 650
])

summary_model.summary()
# ── 파라미터 수 직접 확인 ────────────────────────────────────────────────
total_params = summary_model.count_params()
print(f"\n전체 파라미터 수: {total_params:,}")
print(f"예상 파라미터 수: {784*128+128 + 128*64+64 + 64*10+10:,}")

# 레이어별 파라미터 수 출력
print("\n── 레이어별 파라미터 수 ──")
for layer in summary_model.layers:
    w_count = sum([tf.size(w).numpy() for w in layer.weights])  # 가중치 텐서 크기 합산
    print(f"{layer.name:15s}: {w_count:,}")

## 3. MNIST 손글씨 분류

### 데이터 로드 및 전처리

In [ ]:
# MNIST 데이터셋 로드 (60,000 훈련 / 10,000 테스트)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 픽셀 값을 [0, 255] → [0.0, 1.0] 범위로 정규화
x_train = x_train / 255.0
x_test  = x_test  / 255.0

print("훈련 데이터 shape:", x_train.shape)   # (60000, 28, 28)
print("테스트 데이터 shape:", x_test.shape)   # (10000, 28, 28)
print("레이블 예시:", y_train[:10])

# 샘플 이미지 시각화
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i], cmap='gray')        # 흑백 이미지 표시
    ax.set_title(f"레이블: {y_train[i]}")
    ax.axis('off')
plt.suptitle("MNIST 샘플 이미지", fontsize=14)
plt.tight_layout()
plt.show()

### 모델 구성

In [ ]:
# MNIST 분류 모델 구성
mnist_model = tf.keras.Sequential([
    # 28×28 2D 이미지를 784차원 1D 벡터로 펼침
    tf.keras.layers.Flatten(input_shape=(28, 28), name='flatten'),

    # 은닉층: 128개 뉴런, ReLU 활성화
    tf.keras.layers.Dense(128, activation='relu', name='hidden'),

    # Dropout: 훈련 시 20% 뉴런 무작위 비활성화 → 과적합 방지
    tf.keras.layers.Dropout(0.2, name='dropout'),

    # 출력층: 10개 클래스 (0~9), Softmax로 확률 분포 출력
    tf.keras.layers.Dense(10, activation='softmax', name='output')
], name='mnist_classifier')

mnist_model.summary()

### 학습

In [ ]:
# 모델 컴파일: 옵티마이저, 손실함수, 평가지표 설정
mnist_model.compile(
    optimizer='adam',                          # Adam 옵티마이저 (적응형 학습률)
    loss='sparse_categorical_crossentropy',    # 정수 레이블용 교차 엔트로피 손실
    metrics=['accuracy']                       # 정확도 모니터링
)

# 모델 학습
history = mnist_model.fit(
    x_train, y_train,
    epochs=10,                  # 전체 데이터를 10번 반복 학습
    batch_size=128,             # 미니배치 크기
    validation_split=0.1,       # 훈련 데이터의 10%를 검증 데이터로 사용
    verbose=1                   # 학습 진행 상황 출력
)

### 평가 및 시각화

In [ ]:
# 테스트 세트 최종 평가
test_loss, test_acc = mnist_model.evaluate(x_test, y_test, verbose=0)
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_acc:.4f} ({test_acc*100:.2f}%)")

# ── 학습 곡선 시각화 ─────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 손실 곡선
ax1.plot(history.history['loss'],     label='훈련 손실',   linewidth=2)
ax1.plot(history.history['val_loss'], label='검증 손실',   linewidth=2, linestyle='--')
ax1.set_title('손실 곡선')
ax1.set_xlabel('에포크')
ax1.set_ylabel('손실')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 정확도 곡선
ax2.plot(history.history['accuracy'],     label='훈련 정확도', linewidth=2)
ax2.plot(history.history['val_accuracy'], label='검증 정확도', linewidth=2, linestyle='--')
ax2.set_title('정확도 곡선')
ax2.set_xlabel('에포크')
ax2.set_ylabel('정확도')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('MNIST Sequential 모델 학습 결과', fontsize=14)
plt.tight_layout()
plt.show()

## 정리

| 메서드 | 역할 |
|--------|------|
| `model.compile()` | 손실함수, 옵티마이저, 메트릭 설정 |
| `model.fit()` | 모델 학습 |
| `model.evaluate()` | 테스트 세트 평가 |
| `model.predict()` | 새 데이터 예측 |

**다음**: 02_functional_api.ipynb — 다중 입출력, 잔차 연결